In [1]:
!pip -q install nninteractive SimpleITK huggingface_hub tifffile


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
#IMAGE_PATH = os.path.expanduser("~/Desktop/CK7RY1~U.TIF")
IMAGE_PATH = os.path.expanduser("~/Desktop/CKHRJQ~2.TIF")

In [5]:
# === Cell 3: Multi-lumen membrane segmentation → per-slice "fill inside outer outline" (no circles) + 3D smooth ===
# Saves results to your current working directory (for local Jupyter/VScode use)

import os, time, shutil, gc, numpy as np, SimpleITK as sitk
from scipy.ndimage import gaussian_filter, median_filter, grey_closing, binary_fill_holes, label
from skimage.morphology import (
    ball, disk, binary_opening, binary_closing,
    remove_small_objects, remove_small_holes
)

# ---------------- Knobs (tune as needed) ----------------
OUT_TIF = "lumen_mask_3d.tif"  # output file (in current directory)

# Preprocess
GAUSS_SIGMA       = (1.2, 1.2, 1.2)
DESPECKLE_SIZE    = 3
GREY_CLOSE_RAD    = 1

# Threshold
THRESH_VAL_U8     = 30  # CHANGED: higher starting point after contrast rescale (try 20–40)

# Cleanup before CC
OPEN_RAD_3D       = 2

# Keep lumens (3D)
MIN_SIZE_VOX      = 5000

# Per-slice “fill inside outline”
SL_CLOSE_RAD      = 8
SL_OPEN_RAD       = 0
SL_MIN_OBJECT_2D  = 400
SL_HOLE_AREA_2D   = 500_000

# Final 3D smoothing
FINAL_CLOSE_RAD   = 3
FINAL_OPEN_RAD    = 0
# --------------------------------------------------------

def log(*a): print("[LOG]", *a)

def to_itk_u8(xyz_u8):
    # (X,Y,Z) -> (Z,Y,X)
    return sitk.GetImageFromArray(np.transpose(xyz_u8, (2,1,0)))

t0 = time.time()

# (1) Load & 8-bit
img_itk = sitk.ReadImage(IMAGE_PATH)
arr_zyx = sitk.GetArrayFromImage(img_itk)                    # (Z,Y,X)
arr_xyz = np.transpose(arr_zyx, (2,1,0)).astype(np.float32)  # (X,Y,Z)
X, Y, Z = arr_xyz.shape
vmin, vmax = float(arr_xyz.min()), float(arr_xyz.max())
arr8 = np.clip((arr_xyz - vmin) * (255.0 / max(1e-6, (vmax - vmin))), 0, 255).astype(np.uint8)
del arr_zyx, arr_xyz; gc.collect()
log(f"Loaded membrane channel: shape={X,Y,Z} min/max={vmin:.2f}/{vmax:.2f} → 8-bit")

# (2) Preprocess: Gaussian → median (despeckle) → grey closing
prep = gaussian_filter(arr8.astype(np.float32), sigma=GAUSS_SIGMA)
prep = median_filter(prep, size=DESPECKLE_SIZE)
prep = grey_closing(prep, footprint=ball(GREY_CLOSE_RAD)).astype(np.float32)
log(f"Pre-clean: Gaussian sigma={GAUSS_SIGMA}, Despeckle={DESPECKLE_SIZE}, GreyClose rad={GREY_CLOSE_RAD}")

# (2.5) Contrast (minimal): robust percentile rescale to 2–98% → uint8 0..255  <-- CHANGED (3 lines)
lo, hi = np.percentile(prep, (2.0, 98.0))                                   # CHANGED
prep = (np.clip(prep, lo, hi) - lo) * (255.0 / max(1e-6, (hi - lo)))        # CHANGED
prep = prep.astype(np.uint8); log(f"Contrast: rescale 2–98% -> {prep.min()}/{prep.max()}")  # CHANGED
sitk.WriteImage(to_itk_u8(prep), os.path.abspath("contrast_preview.tif"))

# (3) Fixed threshold
mask = (prep >= float(THRESH_VAL_U8))
del prep, arr8; gc.collect()
log(f"Threshold: value={THRESH_VAL_U8} → voxels={int(mask.sum())}")

# (4) 3D fill holes + opening
mask = binary_fill_holes(mask)
if OPEN_RAD_3D > 0:
    mask = binary_opening(mask, footprint=ball(int(OPEN_RAD_3D)))
log(f"Post-bin: FillHoles=3D, Opening rad={OPEN_RAD_3D} → voxels={int(mask.sum())}")

# (5) 3D connected components, keep ALL ≥ MIN_SIZE_VOX
labels3d, nlab = label(mask)
del mask; gc.collect()
if nlab == 0:
    raise RuntimeError("No objects after thresholding. Lower THRESH_VAL_U8 or adjust smoothing.")
counts = np.bincount(labels3d.ravel()); counts[0] = 0
keep_ids = np.where(counts >= MIN_SIZE_VOX)[0]
if keep_ids.size == 0:
    keep_ids = np.where(counts > 0)[0]
keep_set = set(int(i) for i in keep_ids)
log(f"3D CC: total={nlab}, kept={len(keep_ids)} (≥{MIN_SIZE_VOX} vox)")

# (6) Per-slice, per-lumen: fill inside outline
filled = np.zeros((X, Y, Z), dtype=np.uint8)
se_close = disk(max(1, int(SL_CLOSE_RAD))) if SL_CLOSE_RAD > 0 else None
se_open  = disk(max(1, int(SL_OPEN_RAD)))  if SL_OPEN_RAD  > 0 else None

for z in range(Z):
    lab_z = labels3d[:, :, z]
    if lab_z.max() == 0:
        continue
    present = np.unique(lab_z)
    for lid in present:
        if lid == 0 or lid not in keep_set:
            continue
        sl = (lab_z == lid)
        if se_close is not None:
            sl = binary_closing(sl, footprint=se_close)
        sl = remove_small_objects(sl, min_size=int(SL_MIN_OBJECT_2D))
        sl = binary_fill_holes(sl)
        sl = remove_small_holes(sl, area_threshold=int(SL_HOLE_AREA_2D))
        if se_open is not None:
            sl = binary_opening(sl, footprint=se_open)
        filled[:, :, z] |= sl.astype(np.uint8)

del labels3d; gc.collect()
log("Slice-wise fill-inside-outline complete.")

# (7) Gentle 3D smoothing
bin_mask = filled.astype(bool)
if FINAL_CLOSE_RAD > 0:
    bin_mask = binary_closing(bin_mask, footprint=ball(int(FINAL_CLOSE_RAD)))
if FINAL_OPEN_RAD > 0:
    bin_mask = binary_opening(bin_mask, footprint=ball(int(FINAL_OPEN_RAD)))
final_mask = bin_mask.astype(np.uint8)
del bin_mask, filled; gc.collect()

# (8) Save in current directory
out_path = os.path.abspath(OUT_TIF)
sitk.WriteImage(to_itk_u8(final_mask * 255), out_path, True)
log(f"Saved: {out_path}")
log(f"Done in {time.time()-t0:.2f}s.")


[LOG] Loaded membrane channel: shape=(1024, 1024, 121) min/max=0.00/255.00 → 8-bit
[LOG] Pre-clean: Gaussian sigma=(1.2, 1.2, 1.2), Despeckle=3, GreyClose rad=1
[LOG] Contrast: rescale 2–98% -> 0/255
[LOG] Threshold: value=30 → voxels=7516408
[LOG] Post-bin: FillHoles=3D, Opening rad=2 → voxels=6059160
[LOG] 3D CC: total=281, kept=3 (≥5000 vox)
[LOG] Slice-wise fill-inside-outline complete.
[LOG] Saved: /Users/jenaalsup/embryo-image-segmentation/nninteractive-testing/lumen_mask_3d.tif
[LOG] Done in 155.55s.
